# Reading and Writing Parquet Files
Parquet is a high-density and performance file organization, the following section demonstrates how to with Parquet and import/export it to iguazio database

see more details:  https://arrow.apache.org/docs/python/parquet.html

In [1]:
import pyarrow.parquet as pq
import pandas as pd
import pyarrow as pa

In [2]:
# read some data into a dataframe and show its data & metadata
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', header = None)

# specify columns extracted from wbdc.names
data.columns = ["id","diagnosis","radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean",
                "compactness_mean","concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean",
                "radius_se","texture_se","perimeter_se","area_se","smoothness_se","compactness_se","concavity_se",
                "concave points_se","symmetry_se","fractal_dimension_se","radius_worst","texture_worst",
                "perimeter_worst","area_worst","smoothness_worst","compactness_worst","concavity_worst",
                "concave points_worst","symmetry_worst","fractal_dimension_worst"] 

data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## Write the data into a Parquet on iguazio file system
Note: iguazio file system is mounted under `/v3io/{data-container-name}`

In [5]:
#write the dataframe into a parquet (on iguazio file system)
filepath = '/v3io/users/iguazio/examples/example.parquet'
table = pa.Table.from_pandas(data)
pq.write_table(table, filepath)

In [6]:
# read the parquet into memory and print the head 
table2 = pq.read_table(filepath)
table2.to_pandas().head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## Advanced usage

In [7]:
# read specific columns to save bandwidth/memory and accelerate load
pq.read_table(filepath, columns=['id', 'area_mean']).to_pandas().head()

,id,area_mean
0,842302,1001.0
1,842517,1326.0
2,84300903,1203.0
3,84348301,386.1
4,84358402,1297.0


In [8]:
# read specific columns, if previously written by Pandas (read extra metadata)
pq.read_pandas(filepath, columns=['id', 'area_mean']).to_pandas().head()

,id,area_mean
0,842302,1001.0
1,842517,1326.0
2,84300903,1203.0
3,84348301,386.1
4,84358402,1297.0


In [9]:
parquet_file = pq.ParquetFile(filepath)
parquet_file.schema

id: INT64
diagnosis: BYTE_ARRAY UTF8
radius_mean: DOUBLE
texture_mean: DOUBLE
perimeter_mean: DOUBLE
area_mean: DOUBLE
smoothness_mean: DOUBLE
compactness_mean: DOUBLE
concavity_mean: DOUBLE
concave points_mean: DOUBLE
symmetry_mean: DOUBLE
fractal_dimension_mean: DOUBLE
radius_se: DOUBLE
texture_se: DOUBLE
perimeter_se: DOUBLE
area_se: DOUBLE
smoothness_se: DOUBLE
compactness_se: DOUBLE
concavity_se: DOUBLE
concave points_se: DOUBLE
symmetry_se: DOUBLE
fractal_dimension_se: DOUBLE
radius_worst: DOUBLE
texture_worst: DOUBLE
perimeter_worst: DOUBLE
area_worst: DOUBLE
smoothness_worst: DOUBLE
compactness_worst: DOUBLE
concavity_worst: DOUBLE
concave points_worst: DOUBLE
symmetry_worst: DOUBLE
fractal_dimension_worst: DOUBLE
__index_level_0__: INT64
 

In [10]:
parquet_file.metadata

  created_by: parquet-cpp version 1.5.1-SNAPSHOT
  num_columns: 33
  num_rows: 569
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 8358

In [13]:
# write to multiple partitioned files 
pq.write_to_dataset(table, root_path='my_parq', partition_cols=['radius_mean'])

In [14]:
# read partitioned table
table3 = pq.read_table('my_parq')
table3.to_pandas().head()

,id,diagnosis,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,radius_mean
425,907367,B,21.28,63.19,307.3,0.08117,0.03912,0.002470,0.005159,0.1630,...,28.94,69.92,376.3,0.1126,0.07094,0.01235,0.02579,0.2349,0.08061,10.03
338,897880,B,17.53,64.41,310.8,0.10070,0.07326,0.025110,0.017750,0.1890,...,26.84,71.98,384.0,0.1402,0.14020,0.10550,0.06499,0.2894,0.07664,10.05
185,874158,B,15.11,63.76,317.5,0.09267,0.04695,0.001597,0.002404,0.1703,...,21.18,75.39,437.0,0.1521,0.10190,0.00692,0.01042,0.2933,0.07697,10.08
556,924964,B,19.59,64.73,311.7,0.10030,0.07504,0.005025,0.011160,0.1791,...,22.88,67.88,347.3,0.1265,0.12000,0.01005,0.02232,0.2262,0.06742,10.16
60,858970,B,14.88,64.55,311.9,0.11340,0.08061,0.010840,0.012900,0.2743,...,17.45,69.86,368.6,0.1275,0.09866,0.02168,0.02579,0.3557,0.08020,10.17


In [15]:
# read with multiple threads to accelerate performance
pq.read_table('my_parq', use_threads=4).to_pandas().head()

,id,diagnosis,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,radius_mean
425,907367,B,21.28,63.19,307.3,0.08117,0.03912,0.002470,0.005159,0.1630,...,28.94,69.92,376.3,0.1126,0.07094,0.01235,0.02579,0.2349,0.08061,10.03
338,897880,B,17.53,64.41,310.8,0.10070,0.07326,0.025110,0.017750,0.1890,...,26.84,71.98,384.0,0.1402,0.14020,0.10550,0.06499,0.2894,0.07664,10.05
185,874158,B,15.11,63.76,317.5,0.09267,0.04695,0.001597,0.002404,0.1703,...,21.18,75.39,437.0,0.1521,0.10190,0.00692,0.01042,0.2933,0.07697,10.08
556,924964,B,19.59,64.73,311.7,0.10030,0.07504,0.005025,0.011160,0.1791,...,22.88,67.88,347.3,0.1265,0.12000,0.01005,0.02232,0.2262,0.06742,10.16
60,858970,B,14.88,64.55,311.9,0.11340,0.08061,0.010840,0.012900,0.2743,...,17.45,69.86,368.6,0.1275,0.09866,0.02168,0.02579,0.3557,0.08020,10.17


## Remove Data

In [16]:
# unmark the comment
#!rm -rf $HOME/examples/*